In [1]:
using Random, Gadfly, Optim, Distributions, Distances, LinearAlgebra, ForwardDiff, Cairo

include("../iGMRF.jl");
include("../dataGen.jl");
include("../approxFramework.jl");
include("optim.jl");

In [2]:
# Seed
Random.seed!(400)
# Dimensions de la grille
m₁ = 3;
m₂ = 3;
# Nb total de cellules
m = m₁ * m₂;
# Matrice de structure
κᵤ = 100.0;
F = iGMRF(m₁, m₂, κᵤ);
# Grille cible
grid_target = generateTargetGrid(F);
grid_target[:, :, 1] = grid_target[:, :, 1] .+ 10;
# Génération de données
nobs = 100;
Y = generateData(grid_target, nobs);

In [3]:
res = KLOptim(F, Y)

 * Status: success

 * Candidate solution
    Final objective value:     -2.485120e-06

 * Found with
    Algorithm:     Nelder-Mead

 * Convergence measures
    √(Σ(yᵢ-ȳ)²)/n ≤ 1.0e-08

 * Work counters
    Seconds run:   1  (vs limit Inf)
    Iterations:    161
    f(x) calls:    318


## Analyse de l'approximation

### Visuelle

In [4]:
η̂ = Optim.minimizer(res)[1:m+1];
Σ̂ = diagm(Optim.minimizer(res)[m+2:end].^2);

d̂ = MvNormal(η̂, Σ̂);

Graphiques LOG

In [5]:
npoints = 10000; # Nombre de points sur lesquels évaluer la pdf
x = range(-10, 15, npoints);

# Mode de la posteriori
α = Optim.minimizer(initializeValues(θ -> densityTarget(θ, F=F, Y=Y), [1, fill(0.0, m)...]));

analysisSpace = zeros(m+1, npoints);
for j = 1:npoints
    analysisSpace[:, j] = [x[j], α[2:end]...];
end

simApprox = vec(mapslices(x -> logpdf(d̂, x), analysisSpace, dims=1));
simPosterior = vec(mapslices(x -> densityTarget(x, F=F, Y=Y), analysisSpace, dims=1));

p = plot(
    layer(x=x, y=simApprox, Geom.line, Theme(default_color="red")),
    layer(x=x, y=simPosterior, Geom.line),
    Guide.manual_color_key("Legend", ["Approximation", "Posteriori"], ["red", "deepskyblue"]),
    Theme(background_color="white"),
    Guide.title("Pour kappa_u")
)

draw(PNG("../plots/diag/diag_log_kappa.png"), p)

In [6]:
npoints = 10000; # Nombre de points sur lesquels évaluer la pdf
x = range(-10, 18, npoints);

# Mode de la posteriori
α = Optim.minimizer(initializeValues(θ -> densityTarget(θ, F=F, Y=Y), [1, fill(0.0, m)...]));

analysisSpace = zeros(m+1, npoints);
for j = 1:npoints
    analysisSpace[:, j] = [α[1], x[j], α[3:end]...];
end

simApprox = vec(mapslices(x -> logpdf(d̂, x), analysisSpace, dims=1));
simPosterior = vec(mapslices(x -> densityTarget(x, F=F, Y=Y), analysisSpace, dims=1));

p = plot(
    layer(x=x, y=simApprox, Geom.line, Theme(default_color="red")),
    layer(x=x, y=simPosterior, Geom.line),
    Guide.manual_color_key("Legend", ["Approximation", "Posteriori"], ["red", "deepskyblue"]),
    Theme(background_color="white"),
    Guide.title("Pour mu")
)

draw(PNG("../plots/diag/diag_log_mu.png"), p)

Approx vs MCMC

In [7]:
include("../mcmc.jl");

niter = 10000
δ² = 0.2
κᵤ₀ = 10
μ₀ = zeros(m)

θ = gibbs(niter, Y, m₁=m₁, m₂=m₂, δ²=δ², κᵤ₀=κᵤ₀, μ₀=μ₀);

warming_size = Int(0.2 * niter);
θsampling = θ[warming_size:end, :, :];

In [8]:
# Pour κᵤ

approxDensity(x::Real) = 1/x * pdf(Normal(η̂[1], Σ̂[1, 1]), log(x))
x = 0:0.1:1000

p = plot(
    layer(x=θsampling.value[:, 1, 1], Geom.histogram(density=true)),
    layer(x=x, y=approxDensity.(x), Geom.line, Theme(default_color="red")),
    Guide.manual_color_key("Legend", ["Approximation", "Posteriori"], ["red", "deepskyblue"]),
    Theme(background_color="white"),
    Guide.title("Approx vs MCMC pour kappa_u")
)

draw(PNG("../plots/diag/diag_approxvsmcmc_kappa.png"), p)

In [9]:
# Pour μ

approxDensity(x::Real) = pdf(Normal(η̂[2], Σ̂[2, 2]), x)
x = 9.5:.01:12

p = plot(
    layer(x=θsampling.value[:, 2, 1], Geom.histogram(density=true)),
    layer(x=x, y=approxDensity.(x), Geom.line, Theme(default_color="red")),
    Guide.manual_color_key("Legend", ["Approximation", "Posteriori"], ["red", "deepskyblue"]),
    Theme(background_color="white"),
    Guide.title("Approx vs MCMC pour mu")
)

draw(PNG("../plots/diag/diag_approxvsmcmc_mu.png"), p)

## Valeurs de la divergence en des points donnés

In [36]:
η₀ = α;
Σ₀ = diagm(diag(inv(computeFisherInformation(θ -> densityTarget(θ, F=F, Y=Y), α))));

# x = [1000.0, fill(20.0, m)...];

p = [densityTarget(η₀, F=F, Y=Y)]
q = [logpdf(MvNormal(η₀, Σ₀), η₀)]

kldiv(p, q)

-1.1730904924687093e-6

In [26]:
Σ₀

10×10 Matrix{Float64}:
 0.280608  0.0         0.0         …  0.0         0.0         0.0
 0.0       0.00238073  0.0            0.0         0.0         0.0
 0.0       0.0         0.00197834     0.0         0.0         0.0
 0.0       0.0         0.0            0.0         0.0         0.0
 0.0       0.0         0.0            0.0         0.0         0.0
 0.0       0.0         0.0         …  0.0         0.0         0.0
 0.0       0.0         0.0            0.0         0.0         0.0
 0.0       0.0         0.0            0.00230442  0.0         0.0
 0.0       0.0         0.0            0.0         0.00194701  0.0
 0.0       0.0         0.0            0.0         0.0         0.00231386

In [37]:
exp(p[1])

3.216045878645831e-8